## Tutorial to use OT for evaluating spatiotemporal predictions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from geot.partialot import PartialOT, partial_ot_fixed_locations, partial_ot_unpaired
from geot.plotting import plot_cost_matrix, plot_predictions_and_ground_truth, plot_unpaired_transport_matrix

### Sample data

Let's create some synthetic observations and predictions. We sample the predictions as normally distributed around the true observations.

In [ ]:
NUM_LOCS = 100

In [ ]:
# sample x and y coordinates for NUM_LOCS locations
locations = np.random.rand(NUM_LOCS, 2)
# sample observations at these locations
observations = np.random.normal(size=NUM_LOCS, loc=10, scale=3)
# sample predictions -> add noise to observations
predictions = np.random.normal(size=NUM_LOCS, loc=observations, scale=3)
print("Sampled data", locations.shape, observations.shape, predictions.shape)

In [ ]:
plot_predictions_and_ground_truth(locations, predictions, observations)

### Set cost matrix

To compute the spatial prediction error, we first require a cost matrix. As a simple example, we compute the Euclidean distances between locations.

In [ ]:
# compute pairwise costs
cost_matrix = cdist(locations, locations)
print("C is a 2D matrix of shape", cost_matrix.shape)

# plot matrix
plot_cost_matrix(cost_matrix)

**Note**: The cost matrix does not have to be set to Euclidean distances; it can be really anything - monetary costs, travel times, map-matched distances, etc. Simply replace `cdist` by your function to compute the pairwise distances between locations in another manner

### Compute PartialOT error

We initialize a PartialOT framework. One parameter we need to set is how much to penalize the overall difference between observations and predictions, i.e. between sum(observations) and sum(predictions). Here, we set it to zero since we are only interested in the spatial errors (the transport costs to align predictions and ground truth)

In [ ]:
ot_computer = PartialOT(cost_matrix, penalty_waste=0)

In [ ]:
ot_error = ot_computer(predictions, observations)
print("The OT error is", ot_error.item())

If you only compute it once, there is also a function available that does everything at ones:

In [ ]:
partial_ot_fixed_locations(cost_matrix, predictions, observations, penalty_waste=0)

### OT for unpaired data

So far, we have assumed a use case with *paired* data, i.e., the locations are fixed and there is always one observation and one prediction per location. However, some use cases in GeoAI require to predict the location itself, e.g. where some event occurs. For instance, imagine that someone aims to predict in which streets of a city a crime will occur within the next months. In that case, there is a set of locations where crimes were predicted to occur, and a set of locations where they actually occured, and these sets are unpaired. 

In [ ]:
predicted_locations = np.random.rand(50, 2)
true_locations = np.random.rand(40, 2)

plot_unpaired_transport_matrix(
    predicted_locations, true_locations, np.zeros((50, 40))
)

The function `partial_ot_unpaired` allows to compute the OT transport plan and cost for these predictions:

In [ ]:
ot_error = partial_ot_unpaired(
    predicted_locations, true_locations, cost_matrix=None, import_location=np.array([0, 0]), import_cost_phi=0, return_matrix=False
)
print("OT error for predictions", ot_error)

Again, we can plot the OT matrix which indicates between which locations mass has to be transported:

In [ ]:
# Execute function again, this time with return_matrix=True
transport_matrix = partial_ot_unpaired(
    predicted_locations, true_locations, cost_matrix=None, import_location=np.array([0, 0]), import_cost_phi=0, return_matrix=True
)

In [ ]:
# Plot the transport matrix
plot_unpaired_transport_matrix(
    predicted_locations, true_locations, transport_matrix
)